# Demo Notebook

Steps:
1. Download SAE with SAE Lens.
2. Create a dataset consistent with that SAE. 
3. Fold the SAE decoder norm weights so that feature activations are "correct".
4. Estimate the activation normalization constant if needed, and fold it into the SAE weights.
5. Run the SAE generator for the features you want.

# Set Up

In [ ]:
import torch
from sae_lens import SAE
from transformer_lens import HookedTransformer
from sae_dashboard.sae_vis_data import SaeVisConfig
from sae_dashboard.sae_vis_runner import SaeVisRunner

## Step 1. Download / Initialize SAE

In [ ]:
# For the most part I'll try to import functions and classes near where they are used
# to make it clear where they come from.

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

model = HookedTransformer.from_pretrained("gemma-2b", device=device, dtype="bfloat16")

# the cfg dict is returned alongside the SAE since it may contain useful information for analysing the SAE (eg: instantiating an activation store)
# Note that this is not the same as the SAEs config dict, rather it is whatever was in the HF repo, from which we can extract the SAE config dict
# We also return the feature sparsities which are stored in HF for convenience.
sae, cfg_dict, sparsity = SAE.from_pretrained_with_cfg_and_sparsity(
    release="gemma-2b-res-jb",  # see other options in sae_lens/pretrained_saes.yaml
    sae_id="blocks.6.hook_resid_post",  # won't always be a hook point
    device=device,
)
# fold w_dec norm so feature activations are accurate
sae.fold_W_dec_norm()

# 2. Get token dataset

In [ ]:
from sae_lens import ActivationsStore

activations_store = ActivationsStore.from_sae(
    model=model,
    sae=sae,
    streaming=True,
    store_batch_size_prompts=16,
    n_batches_in_buffer=8,
    device=device,
    dataset=sae.cfg.metadata.dataset_path,
)

# Some SAEs will require we estimate the activation norm and fold it into the weights. This is easy with SAE Lens.
# TODO: figure out what to do with this
if sae.cfg.normalize_activations == "expected_average_only_in":
    norm_scaling_factor = activations_store.estimate_norm_scaling_factor(
        n_batches_for_norm_estimate=30
    )
    sae.fold_activation_norm_scaling_factor(norm_scaling_factor)

In [ ]:

from sae_dashboard.utils_fns import get_tokens

# 1000 prompts is plenty for a demo.
token_dataset = get_tokens(activations_store, 128)

# Step 3 Evaluate the SAE

In [ ]:
from sae_lens import run_evals
from sae_lens.evals import EvalConfig
from sae_lens.training.activation_scaler import ActivationScaler

eval_config = EvalConfig(
    batch_size_prompts=8,
    n_eval_reconstruction_batches=3,
    compute_ce_loss=True,
)

scalar_metrics, feature_metrics = run_evals(
    sae=sae,
    activation_store=activations_store,
    model=model,
    activation_scaler=ActivationScaler(),
    eval_config=eval_config,
)

# CE Loss score should be high for residual stream SAEs
print(scalar_metrics["model_performance_preservation"]["ce_loss_score"])

# ce loss without SAE should be fairly low < 3.5 suggesting the Model is being run correctly
print(scalar_metrics["model_performance_preservation"]["ce_loss_without_sae"])

# ce loss with SAE shouldn't be massively higher
print(scalar_metrics["model_performance_preservation"]["ce_loss_with_sae"])

# 4. Generate Feature Dashboards

In [ ]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
from pathlib import Path

test_feature_idx_gpt = list(range(256))

feature_vis_config_gpt = SaeVisConfig(
    hook_point=sae.cfg.metadata.hook_name,
    features=test_feature_idx_gpt,
    minibatch_size_features=64,
    minibatch_size_tokens=256,  # this is number of prompts at a time.
    verbose=True,
    device=device,  # Use the same device as the model
    cache_dir=Path(
        "demo_activations_cache"
    ),  # this will enable us to skip running the model for subsequent features.
    dtype="bfloat16",
)

data = SaeVisRunner(feature_vis_config_gpt).run(
    encoder=sae,  # type: ignore
    model=model,
    tokens=token_dataset,
)

In [ ]:
from sae_dashboard.data_writing_fns import save_feature_centric_vis

filename = f"demo_feature_dashboards.html"
save_feature_centric_vis(sae_vis_data=data, filename=filename)